In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import json
from PIL import Image
import pprint

/data/lixiang10/anaconda3/envs/cogact/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-07 19:34:14.504642: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-07 19:34:14.504705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-07 19:34:14.506113: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-07 19:34:14.514376: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorF

In [2]:
# Load the LIBERO reasonings
with open("/data/lixiang10/embodiedCoT/embodied_features_bridge.json", "r") as f:
    reasoning_data = json.load(f)

In [3]:
print("len bridge_reasoning files:", len(reasoning_data.keys()))

for filename in reasoning_data.keys():
    print(f"filename: {filename}")
    for demo_id in reasoning_data[filename].keys():
        demo = reasoning_data[filename][demo_id]
        print(f"  Demo ID: {demo_id}")
        print(demo.keys())
        break
    break

len bridge_reasoning files: 3200
filename: /nfs/kun2/users/homer/datasets/bridge_data_all/numpy_256/bridge_data_v2/deepthought_folding_table/stack_blocks/19/train/out.npy
  Demo ID: 43
dict_keys(['features', 'metadata', 'reasoning'])


In [4]:
print(demo['features'])
print(demo['metadata'])
print(demo['reasoning'])

{'move_primitive': ['stop', 'stop', 'move up', 'move up', 'move up', 'move right up', 'move right up', 'move right', 'move right', 'move right', 'move right', 'move right', 'move right', 'stop', 'stop', 'stop', 'stop', 'stop', 'close gripper', 'close gripper', 'close gripper', 'close gripper', 'close gripper', 'stop', 'stop', 'move left', 'move left', 'move left down', 'move left down', 'move down', 'move down, open gripper', 'open gripper', 'open gripper', 'open gripper', 'open gripper', 'move up', 'move up', 'move up', 'stop', 'stop'], 'gripper_position': [[97, 45], [97, 45], [89, 52], [83, 58], [82, 57], [84, 55], [85, 55], [85, 55], [85, 55], [85, 55], [85, 55], [85, 55], [85, 55], [85, 55], [152, 89], [152, 89], [152, 89], [152, 89], [152, 89], [152, 89], [152, 89], [152, 89], [152, 89], [153, 88], [153, 88], [163, 79], [163, 79], [173, 52], [173, 52], [173, 43], [177, 31], [178, 28], [179, 29], [176, 30], [178, 30], [174, 32], [167, 36], [160, 40], [145, 46], [130, 53]], 'bboxes'

In [50]:
subtask_keyframe = []
move_keyframe = []
vision_keyframe = []
cur_subtask, cur_move = '', ''
cur_vision = set()

for key, value in demo['reasoning'].items():
    move = value['move']
    subtask = value['subtask']
    vision = demo['features']['bboxes'][int(key)]

    #print(move)
    if move != cur_move:
        cur_move = move
        move_keyframe.append(1)
    else:
        move_keyframe.append(0)

    #print(subtask)
    if subtask != cur_subtask:
        cur_subtask = subtask
        subtask_keyframe.append(1)
    else:
        subtask_keyframe.append(0)

    visble_objs_set = set()
    # print(vision)
    if len(vision) == 0:
        vision_keyframe.append(0)
        continue
    for obj in vision:
        for info in obj:
            if type(info) is str:
                visble_objs_set.add(info)
    #print(visble_objs_set)
    if visble_objs_set != cur_vision:
        cur_vision = visble_objs_set
        vision_keyframe.append(1)
    else:
        vision_keyframe.append(0)

print("Subtask keyframe:", subtask_keyframe)
print("Move keyframe:", move_keyframe)
print("Vision keyframe:", vision_keyframe)


Subtask keyframe: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
Move keyframe: [1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0]
Vision keyframe: [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]


In [52]:
demo['features']['subtask_keyframe'] = subtask_keyframe
demo['features']['move_keyframe'] = move_keyframe
demo['features']['vision_keyframe'] = vision_keyframe

In [53]:
print(len(demo['features']['move_primitive']))
print(demo['features']['move_primitive'])
print(len(demo['features']['gripper_position']))
print(demo['features']['gripper_position'])
print(len(demo['features']['bboxes']))
print(demo['features']['bboxes'])
print(len(demo['features']['subtask_keyframe']))
print(demo['features']['subtask_keyframe'])
print(len(demo['features']['move_keyframe']))
print(demo['features']['move_keyframe'])
print(len(demo['features']['vision_keyframe']))
print(demo['features']['vision_keyframe'])

40
['stop', 'stop', 'move up', 'move up', 'move up', 'move right up', 'move right up', 'move right', 'move right', 'move right', 'move right', 'move right', 'move right', 'stop', 'stop', 'stop', 'stop', 'stop', 'close gripper', 'close gripper', 'close gripper', 'close gripper', 'close gripper', 'stop', 'stop', 'move left', 'move left', 'move left down', 'move left down', 'move down', 'move down, open gripper', 'open gripper', 'open gripper', 'open gripper', 'open gripper', 'move up', 'move up', 'move up', 'stop', 'stop']
40
[[97, 45], [97, 45], [89, 52], [83, 58], [82, 57], [84, 55], [85, 55], [85, 55], [85, 55], [85, 55], [85, 55], [85, 55], [85, 55], [85, 55], [152, 89], [152, 89], [152, 89], [152, 89], [152, 89], [152, 89], [152, 89], [152, 89], [152, 89], [153, 88], [153, 88], [163, 79], [163, 79], [173, 52], [173, 52], [173, 43], [177, 31], [178, 28], [179, 29], [176, 30], [178, 30], [174, 32], [167, 36], [160, 40], [145, 46], [130, 53]]
40
[[[0.3405100107192993, 'wooden blocks', 

In [66]:
from tqdm import tqdm
for filename in tqdm(reasoning_data.keys()):
    for demo_id in reasoning_data[filename].keys():
        demo = reasoning_data[filename][demo_id]
        subtask_keyframe = []
        move_keyframe = []
        vision_keyframe = []
        cur_subtask, cur_move = '', ''
        cur_vision = set()
        if 'reasoning' not in demo.keys():
            continue
        for key, value in demo['reasoning'].items():
            move = value['move']
            subtask = value['subtask']
            if 'bboxes' in demo['features']  and int(key) < len(demo['features']['bboxes']):
                vision = demo['features']['bboxes'][int(key)]
            else:
                vision = []
            #print(move)
            if move != cur_move:
                cur_move = move
                move_keyframe.append(1)
            else:
                move_keyframe.append(0)

            #print(subtask)
            if subtask != cur_subtask:
                cur_subtask = subtask
                subtask_keyframe.append(1)
            else:
                subtask_keyframe.append(0)

            visble_objs_set = set()
            # print(vision)
            if len(vision) == 0:
                vision_keyframe.append(0)
                continue
            for obj in vision:
                for info in obj:
                    if type(info) is str:
                        visble_objs_set.add(info)
            #print(visble_objs_set)
            if visble_objs_set != cur_vision:
                cur_vision = visble_objs_set
                vision_keyframe.append(1)
            else:
                vision_keyframe.append(0)
        demo['features']['subtask_keyframe'] = subtask_keyframe
        demo['features']['move_keyframe'] = move_keyframe
        demo['features']['vision_keyframe'] = vision_keyframe

100%|██████████| 3200/3200 [00:06<00:00, 471.58it/s] 


In [68]:
demo['features'].keys()

dict_keys(['move_primitive', 'gripper_position', 'bboxes', 'subtask_keyframe', 'move_keyframe', 'vision_keyframe'])

In [69]:
print(demo['features']['move_primitive'])
print(demo['features']['move_keyframe'])

['stop', 'move down', 'stop', 'move down', 'stop', 'move down', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'close gripper', 'close gripper', 'close gripper', 'close gripper', 'open gripper', 'move up', 'move up', 'move up', 'move forward', 'move forward', 'move forward', 'move forward down', 'move down', 'move down, open gripper', 'open gripper', 'open gripper', 'open gripper', 'move up, open gripper', 'move up', 'move up', 'stop', 'stop', 'stop', 'stop', 'move left', 'move left', 'move left', 'move left', 'move left', 'move backward', 'move backward', 'move backward', 'move backward', 'move down', 'move down', 'move down', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'stop', 'close gripper', 'close gripper', 'close gripper', 'close gripper', 'close gripper', 'stop', 'move up', 'move up', 'move up', 'move forward up', 'move forward', 'move forward', 'move forward', 'move forward down', 'move down, open gripper', 'open gripper', 'open gripper', 'open gripper', 'open grip

In [70]:
with open("/data/lixiang10/embodiedCoT/embodied_features_bridge_keyframe.json", "w") as f:
    json.dump(reasoning_data, f, indent=4)

In [71]:
with open("/data/lixiang10/embodiedCoT/embodied_features_bridge_keyframe.json", "r") as f:
    reasoning_data_keyframe = json.load(f)

In [74]:
print("len bridge_reasoning files:", len(reasoning_data_keyframe.keys()))

for filename in reasoning_data_keyframe.keys():
    print(f"filename: {filename}")
    for demo_id in reasoning_data_keyframe[filename].keys():
        demo_keyframe = reasoning_data_keyframe[filename][demo_id]
        print(f"  Demo ID: {demo_id}")
        print(demo_keyframe.keys())
        break
    break

len bridge_reasoning files: 3200
filename: /nfs/kun2/users/homer/datasets/bridge_data_all/numpy_256/bridge_data_v2/deepthought_folding_table/stack_blocks/19/train/out.npy
  Demo ID: 43
dict_keys(['features', 'metadata', 'reasoning'])


In [75]:
print(demo_keyframe['features']['move_primitive'])
print(demo_keyframe['features']['move_keyframe'])
print(demo_keyframe['features']['subtask_keyframe'])

['stop', 'stop', 'move up', 'move up', 'move up', 'move right up', 'move right up', 'move right', 'move right', 'move right', 'move right', 'move right', 'move right', 'stop', 'stop', 'stop', 'stop', 'stop', 'close gripper', 'close gripper', 'close gripper', 'close gripper', 'close gripper', 'stop', 'stop', 'move left', 'move left', 'move left down', 'move left down', 'move down', 'move down, open gripper', 'open gripper', 'open gripper', 'open gripper', 'open gripper', 'move up', 'move up', 'move up', 'stop', 'stop']
[1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]


In [76]:
for key, value in demo_keyframe['reasoning'].items():

    print(f"value: {value['subtask']}")

value: Reach for the wooden arch.
value: Reach for the wooden arch.
value: Reach for the wooden arch.
value: Reach for the wooden arch.
value: Reach for the wooden arch.
value: Reach for the wooden arch.
value: Reach for the wooden arch.
value: Reach for the wooden arch.
value: Reach for the wooden arch.
value: Reach for the wooden arch.
value: Reach for the wooden arch.
value: Reach for the wooden arch.
value: Reach for the wooden arch.
value: Reach for the wooden arch.
value: Grasp the wooden arch.
value: Grasp the wooden arch.
value: Grasp the wooden arch.
value: Grasp the wooden arch.
value: Grasp the wooden arch.
value: Grasp the wooden arch.
value: Grasp the wooden arch.
value: Grasp the wooden arch.
value: Grasp the wooden arch.
value: Move the wooden arch to the table.
value: Move the wooden arch to the table.
value: Move the wooden arch to the table.
value: Move the wooden arch to the table.
value: Move the wooden arch to the table.
value: Move the wooden arch to the table.
va